In [1]:
!pip install -r ./requirements.txt -q


[notice] A new release of pip is available: 23.3 -> 23.3.1
[notice] To update, run: pip install --upgrade pip


In [2]:
!pip show langchain

Name: langchain
Version: 0.0.342
Summary: Building applications with LLMs through composability
Home-page: https://github.com/langchain-ai/langchain
Author: 
Author-email: 
License: MIT
Location: /workspace/.pyenv_mirror/user/current/lib/python3.12/site-packages
Requires: aiohttp, anyio, dataclasses-json, jsonpatch, langchain-core, langsmith, numpy, pydantic, PyYAML, requests, SQLAlchemy, tenacity
Required-by: 


In [3]:
import os
from dotenv import load_dotenv, find_dotenv
load_dotenv(find_dotenv(),override=True)

os.environ.get('PINECONE_API_KEY')

'f6fae977-8607-404c-8f93-d1f9cab4f17a'

## LLM Models (Wrappers): GPT3

In [4]:
from langchain.llms import OpenAI
llm = OpenAI(model_name='text-davinci-003', temperature=.7, max_tokens=512)
print(llm)

OpenAI
Params: {'model_name': 'text-davinci-003', 'temperature': 0.7, 'top_p': 1, 'frequency_penalty': 0, 'presence_penalty': 0, 'n': 1, 'logit_bias': {}, 'max_tokens': 512}


In [5]:
output = llm('explain quantum mechanics in one sentance')
print (output)



Quantum mechanics is a physical theory that describes the behavior of matter and energy at the scale of atoms and subatomic particles.


In [6]:
print (llm.get_num_tokens('explain quantum mechanics in one sentance'))

8


In [7]:
output= llm.generate(['....is the capital of framce', "what is the formula for the area of a circle"])

In [8]:
print (output.generations)

[[Generation(text='\nParis', generation_info={'finish_reason': 'stop', 'logprobs': None})], [Generation(text='\n\nThe formula for the area of a circle is A = πr², where A is the area, π is the constant pi (3.14159...), and r is the radius of the circle.', generation_info={'finish_reason': 'stop', 'logprobs': None})]]


In [9]:
print (output.generations[1][0].text)



The formula for the area of a circle is A = πr², where A is the area, π is the constant pi (3.14159...), and r is the radius of the circle.


In [10]:
len(output.generations)

2

In [11]:
foroutput = llm.generate(["write an original tagline for a burger resturatint"] * 3)

In [12]:
for o in output.generations:
    print (o[0].text, end='')


Paris

The formula for the area of a circle is A = πr², where A is the area, π is the constant pi (3.14159...), and r is the radius of the circle.

#### ChatModels: GPT-3.5-Turbo and GPT-4

TODO: https://platform.openai.com/docs/guides/text-generation/chat-completions-api
Read more about what the different roles are

In [13]:
from langchain.schema import(
    AIMessage,
    HumanMessage,
    SystemMessage
)

from langchain.chat_models import ChatOpenAI


In [14]:
chat = ChatOpenAI(model_name= 'gpt-3.5-turbo', temperature=0.5, max_tokens=1024)
messages = [
    SystemMessage(content="you are a physicist and only respond in german"),
    HumanMessage(content="explain quantum mechanics in one sentance")
]
output = chat(messages)

In [15]:
print(output.content)

Quantenmechanik ist das physikalische Modell, das das Verhalten von Teilchen und Systemen auf mikroskopischer Ebene beschreibt.


### Prompt Templates

In [16]:
from langchain import PromptTemplate

In [17]:
template = '''You are an experianced virologist.
Write a few sentances about the following {virus} in {language}.'''

prompt= PromptTemplate(
    input_variables=['virus','language'],
    template=template
)

print (prompt)

input_variables=['language', 'virus'] template='You are an experianced virologist.\nWrite a few sentances about the following {virus} in {language}.'


In [18]:
from langchain.llms import OpenAI
llm = OpenAI(model_name='text-davinci-003', temperature=.7)

output = llm(prompt.format(virus="ebola",language='english'))
print (output)



Ebola is a highly contagious virus that can cause severe illness and even death in humans. It is spread through contact with the blood or bodily fluids of an infected person, or contact with objects that have been contaminated with the virus. Symptoms of Ebola include fever, headache, muscle pain, fatigue, and vomiting. Left untreated, the virus can cause internal bleeding, organ failure, and death.


#### Simple Chains

In [19]:
from langchain.chat_models import ChatOpenAI
from langchain import PromptTemplate
from langchain.chains import LLMChain

llm = OpenAI(model_name='gpt-3.5-turbo', temperature=.5)

template = '''You are an experianced virologist.
Write a few sentances about the following {virus} in {language}.'''

prompt= PromptTemplate(
    input_variables=['virus','language'],
    template=template
)

chain = LLMChain(llm=llm,prompt=prompt)
output = chain.run({"virus": "HSV","language":"french"})

/workspace/.pyenv_mirror/user/current/lib/python3.12/site-packages/langchain/llms/openai.py:244: UserWarning: You are trying to use a chat model. This way of initializing it is no longer supported. Instead, please use: `from langchain.chat_models import ChatOpenAI`
  warnings.warn(
/workspace/.pyenv_mirror/user/current/lib/python3.12/site-packages/langchain/llms/openai.py:1039: UserWarning: You are trying to use a chat model. This way of initializing it is no longer supported. Instead, please use: `from langchain.chat_models import ChatOpenAI`
  warnings.warn(


In [20]:
print(output)

L'HSV, ou le virus de l'herpès simplex, est un virus à ADN de la famille des Herpesviridae. Il existe deux types d'HSV, le HSV-1 et le HSV-2, qui peuvent provoquer des infections génitales et orales. Ces infections sont caractérisées par des lésions cutanées vésiculaires douloureuses et récurrentes. La transmission de l'HSV se fait principalement par contact direct avec les lésions ou par voie sexuelle.
